In [1]:
from pyce import raster as pyraster 
from pyce import shape as pyshape
import pyce

In [2]:
import ee, geemap, eemont
import rioxarray
import geopandas as gpd
from geopandas import GeoSeries as Gs
import pandas as pd
import os

import folium

In [3]:
ee.Authenticate()
ee.Initialize()

-----------------------------------------

RGI 

In [21]:
# FC
# Europe RGI Boundaries
geom_RGI_2000=ee.FeatureCollection('projects/sat-io/open-datasets/RGI/RGI_VECTOR_MERGED_V7')
geom_RGI_2015=ee.FeatureCollection('users/aguerou/ice_and_life/carto_h1b/lia_shp/c3s_gi_rgi11_s2_2015_v2')
# FR
RGI_2000_FR = ee.FeatureCollection('projects/ee-roniritzganem/assets/EU_GL/RGI_FR/RGI_2000_FR')
RGI_2015_FR = ee.FeatureCollection('projects/ee-roniritzganem/assets/EU_GL/RGI_FR/RGI_2015_FR')

In [22]:
# GDF
def fc_to_gdf(fc):
    return ee.data.computeFeatures({
        'expression': fc,
        'fileFormat': 'GEOPANDAS_GEODATAFRAME'
    })

gdf_geom_RGI_2000 = fc_to_gdf(geom_RGI_2000)
gdf_geom_RGI_2015 = fc_to_gdf(geom_RGI_2015)
gdf_RGI_2000_FR = fc_to_gdf(RGI_2000_FR)
gdf_RGI_2015_FR = fc_to_gdf(RGI_2015_FR)


In [ ]:
gdf_RGI_2015_FR

ts and geometry import

In [4]:
# path to local folder glacier
folder = "C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1"

glacier_raster = {
    year: rioxarray.open_rasterio(f"{folder}/GLACIER_{year}.tif")
    for year in range(1985, 2025) 
}

RasterioIOError: C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1/GLACIER_1985.tif: No such file or directory

In [ ]:
# path to local folder water
folder = "C:/Users/ronir/Desktop/Google_Earth_Engine/Export/Vectorisation_V1"


water_rasters = {
    year: rioxarray.open_rasterio(f"{folder}/WATER_{year}.tif")
    for year in range(1985, 2025)
}


In [ ]:
EU_lia = '' # add LIA

In [10]:
gdf_lia_vdef=gpd.read_file("C:/Users/ronir/Desktop/Stage_Carrtel_2025/QGIS/Shape LIA/transfer_9869808_files_c7ca226a/Glaciers_1850_final_mourey2025.shp")

-----------------------------------------

# FUNCTIONS 

In [9]:
def add_year_column(gdf, year):
    """
    Ajoute une colonne 'année' à un GeoDataFrame, remplie avec une année spécifique.
    
    Args:
        gdf (geopandas.GeoDataFrame): Le GeoDataFrame d'entrée.
        year (int): L'année à ajouter dans la nouvelle colonne 'année'.
        
    Returns:
        geopandas.GeoDataFrame: Le GeoDataFrame avec la colonne 'année' ajoutée.
    """
    gdf['year'] = year
    return gdf

In [19]:
def multi_raster_to_gdf(rasters,tresh_value):
    """
    Polygonise plusieurs couches raster (par année) et retourne un GeoDataFrame concaténé.
    
    Paramètres:
        rasters (xarray.Dataset): Dataset avec une variable par année (nommée par année ou identifiant temporel).
    
    Retour:
        GeoDataFrame: géométries polygonisées concaténées avec colonne 'year'.
    """
    gdf_list = []

    for year_str, raster in rasters.items():
        # Convertir l'identifiant en année (assume que c’est un str représentant l’année, sinon adapter)
        year = int(str(year_str))

        # Polygonisation
        shape = pyraster.polygonize_raster(
            raster,
            mask=(raster == tresh_value),
            transform=raster.rio.transform()
        )

        # GeoDataFrame + ajout de la colonne année
        gdf = gpd.GeoDataFrame(
            geometry=[shape],
            crs=raster.rio.crs
        ).explode(ignore_index=True)
        gdf["year"] = year

        gdf_list.append(gdf)

    # Concaténer tous les GeoDataFrames
    gdf_all = pd.concat(gdf_list, ignore_index=True)

    return gdf_all

In [11]:
def filter_water_by_glacier_gpd(water_to_filter, glacier_to_filter):
    """
    Filtre les géométries des eaux à l'intérieur et à l'extérieur des glaciers sur plusieurs années,
    et retourne deux GeoDataFrames concaténés avec une colonne 'year'.

    Args:
        water_to_filter (dict[int, geopandas.GeoDataFrame]): GeoDataFrames d'eau par année.
        glacier_to_filter (dict[int, geopandas.GeoDataFrame]): GeoDataFrames de glaciers par année.

    Returns:
        tuple:
            - GeoDataFrame des eaux dans les glaciers (avec colonne 'year')
            - GeoDataFrame des eaux hors glaciers (avec colonne 'year')
    """
    water_in_list = []
    water_out_list = []

    for year in range(1984, 2025):
        
        gdf_water = water_to_filter[water_to_filter["year"]==year]
        gdf_glacier = glacier_to_filter[glacier_to_filter["year"]==year]

        # Calculer les géométries à l'intérieur des glaciers
        gdf_in = gdf_water.sjoin(gdf_glacier, how='inner', predicate='within')

        #Calculer celles en dehors des glaciers
        gdf_out = gdf_water.loc[~gdf_water.index.isin(gdf_in.index)]

        water_in_list.append(gdf_in)
        water_out_list.append(gdf_out)

    # Concaténer tous les résultats en deux GeoDataFrames
    water_in_glacier = pd.concat(water_in_list, ignore_index=True)
    water_outside = pd.concat(water_out_list, ignore_index=True)

    return water_in_glacier, water_outside

In [12]:
def exclude_water_inside_geometry(water_gdf,geom):
    """
    Exclut les polygones de water_gdf entièrement contenus dans geometry.
    """
    water_in_list = []
    water_out_list = []
       
    water_in_geom = water_gdf.sjoin(geom, how='inner', predicate='within')
    water_outside_geom = water_gdf.loc[~water_gdf.index.isin(water_in_geom.index)]

    water_in_list.append(water_in_geom)
    water_out_list.append(water_outside_geom )

    water_in_geom_pd = pd.concat(water_in_list, ignore_index=True)
    water_outside_geom_pd = pd.concat(water_out_list, ignore_index=True)

    return  water_outside_geom_pd
    

In [13]:
def exclude_inside_RGI_water(gdf, rgi_gdf,year_rgi):
    """
    Exclut les polygones de water_gdf entièrement contenus dans rgi_gdf
    uniquement si leur année est inférieure ou égale à celle du RGI.
    
    Args:
        water_gdf (GeoDataFrame): GeoDataFrame des plans d'eau avec une colonne 'année'.
        rgi_gdf (GeoDataFrame): GeoDataFrame des glaciers avec une colonne 'année'.

    Returns:
        GeoDataFrame: GeoDataFrame des plans d'eau filtré.
    """
    water_in_rgi_list=[]
    water_outside_RGI_list=[]
    
    for year in range(1984, year_rgi+1):

        gdf_to_filter = gdf[gdf["year"]==year]
        
        # Jointure spatiale
        gdf_in = gdf_to_filter.sjoin(rgi_gdf, how='inner', predicate='within')
        gdf_out = gdf_to_filter.loc[~gdf_to_filter.index.isin(gdf_in.index)]

        water_outside_RGI_list.append(gdf_out)
        water_outside_RGI = pd.concat(water_outside_RGI_list, ignore_index=True)
        
    return  water_outside_RGI
    

In [14]:
def exclude_outside_RGI_glacier(gdf, rgi_gdf,year_rgi):
    """
    Exclut les polygones de glacier_gdf entièrement non contenus dans rgi_gdf
    uniquement si leur année est inférieure ou égale à celle du RGI.
    
    Args:
        glacier_gdf (GeoDataFrame): GeoDataFrame des plans d'eau avec une colonne 'année'.
        rgi_gdf (GeoDataFrame): GeoDataFrame des glaciers avec une colonne 'année'.

    Returns:
        GeoDataFrame: GeoDataFrame des plans d'eau filtré.
    """
    glacier_inside_rgi_list=[]
    glacier_outside_rgi_list=[]
    
    for year in range(1984, year_rgi+1):

        gdf_to_filter = gdf[gdf["year"]==year]
        
        # Jointure spatiale
        gdf_in = gdf_to_filter.sjoin(rgi_gdf, how='inner', predicate='intersects')
        gdf_to_conserve = gdf_to_filter.loc[gdf_to_filter.index.isin(gdf_in.index)]

        glacier_inside_rgi_list.append(gdf_to_conserve)
        glacier_inside_rgi = pd.concat(glacier_inside_rgi_list, ignore_index=True)
        
    return  glacier_inside_rgi
    

Test (filter 5 years)

In [15]:
# Filtering with 5 years
def water_time_filtering_propre(water_gdf):
    """
    Filtre les polygones d'eau pour ne conserver que ceux qui s'intersectent 
    sur plusieurs années **futures** consécutives (croissant), de 1984 à 2024.

    - 1984 à 2020 : intersection sur 5 années (année, +1, +2, +3, +4)
    - 2021 : intersection sur 4 années (année, +1, +2, +3)
    - 2022 : intersection sur 3 années (année, +1, +2)
    - 2023 : intersection sur 2 années (année, +1)
    - 2024 : utilisée seule (aucun test)
    """
    start_year=1984
    end_year=2024
    years = range(start_year, end_year+1)
    gdf_time_filtered_list = []

    for year in years:
        # Nombre d'années à considérer
        dy = min(5, end_year - year + 1)
       
        gdf_base = water_gdf[water_gdf['year'] == year]
        gdf_future = water_gdf[(water_gdf['year'] > year) & (water_gdf['year'] <= year + dy)]

        if dy == 1:
            # Cas 1984, on garde tout
            filtered = gdf_base
            gdf_time_filtered_list.append(filtered)
        else: 
            joined = gdf_base.sjoin(gdf_future, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_future['year'].unique())
            
            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom == expected_years].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered

In [17]:
def glacier_time_filtering_propre(glacier_gdf):
    """
    Filtre les polygones de glacier pour ne conserver que ceux qui s'intersectent 
    sur toutes les années passées consécutives (décroissant), de 2024 à 1984.

    - 2024 à 1988 : intersection sur 5 années (année, -1, -2, -3, -4)
    - 1987 : intersection sur 4 années (année, -1, -2, -3)
    - 1986 : intersection sur 3 années (année, -1, -2)
    - 1985 : intersection sur 2 années (année, -1)
    - 1984 : utilisée seule (aucun test)
    """
    start_year = 1985
    end_year = 2024
    gdf_time_filtered_list = []

    for year in range(end_year, start_year - 1, -1):
        dy = min(5, year - start_year + 1)

        gdf_base = glacier_gdf[glacier_gdf['year'] == year]
        gdf_past = glacier_gdf[(glacier_gdf['year'] < year) & (glacier_gdf['year'] >= year - dy)]

        if dy == 1:
            gdf_time_filtered_list.append(gdf_base)
        else:
            # Jointure spatiale
            joined = gdf_base.sjoin(gdf_past, how='inner', predicate='intersects')
            
            joined['index_left'] = joined.index
            
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_past['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom == expected_years].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered.sort_values(by='year', ascending=True).reset_index(drop=True)

Filtering

In [16]:
# Each polygone has to be detected 80% of the years since its first detection
# Efficace à l'échelle des alpes sans besoin de clip au lac, pollution restante en haut des domes, 

def water_time_filtering_propre_total(water_gdf):
    """
    Filtre les polygones d'eau pour ne conserver que ceux qui s'intersectent 
    sur plusieurs années **futures** consécutives (croissant), de 1984 à 2024.

    - 1984 à 2020 : intersection sur 5 années (année, +1, +2, +3, +4)
    - 2021 : intersection sur 4 années (année, +1, +2, +3)
    - 2022 : intersection sur 3 années (année, +1, +2)
    - 2023 : intersection sur 2 années (année, +1)
    - 2024 : utilisée seule (aucun test)
    """
    end_year=2024
    start_year=1984
    years = range(start_year, end_year+1)
    gdf_time_filtered_list = []

    for year in years:
        # Nombre d'années à considérer
        dy = end_year - year + 1
       
        gdf_base = water_gdf[water_gdf['year'] == year]
        gdf_future = water_gdf[(water_gdf['year'] > year) & (water_gdf['year'] <= year + dy)]

        if dy == 1:
            # Cas 1984, on garde tout
            filtered = gdf_base
            gdf_time_filtered_list.append(filtered)
        else: 
            joined = gdf_base.sjoin(gdf_future, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_future['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom.apply(lambda years: len(years) >= 0.8 * len(expected_years))].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered

In [18]:
def glacier_time_filtering_propre_total(glacier_gdf):
    """
    Filtre les polygones de glacier pour ne conserver que ceux qui s'intersectent 
    sur toutes les années passées consécutives (décroissant), de 2024 à 1984.

    - 2024 à 1988 : intersection sur 5 années (année, -1, -2, -3, -4)
    - 1987 : intersection sur 4 années (année, -1, -2, -3)
    - 1986 : intersection sur 3 années (année, -1, -2)
    - 1985 : intersection sur 2 années (année, -1)
    - 1984 : utilisée seule (aucun test)
    """
    start_year = 1985
    end_year = 2024
    gdf_time_filtered_list = []

    for year in range(end_year, start_year - 1, -1):
        dy =  year - start_year + 1

        gdf_base = glacier_gdf[glacier_gdf['year'] == year]
        gdf_past = glacier_gdf[(glacier_gdf['year'] < year) & (glacier_gdf['year'] >= year - dy)]

        if dy == 1:
            gdf_time_filtered_list.append(gdf_base)
        else:
            # Jointure spatiale
            joined = gdf_base.sjoin(gdf_past, how='inner', predicate='intersects')
            joined['index_left'] = joined.index
            # Années réellement intersectées dans cette plage
            expected_years = set(gdf_past['year'].unique())

            # Pour chaque géométrie de gdf_base, collecter les années qu'elle intersecte
            years_by_geom = joined.groupby('index_left')['year_right'].apply(set)

            # Garder seulement les géométries qui intersectent toutes les années attendues
            valid_indices = years_by_geom[years_by_geom.apply(lambda years: len(years) >= 0.8 * len(expected_years))].index

            # Filtrage
            filtered = gdf_base.loc[gdf_base.index.isin(valid_indices)]
            gdf_time_filtered_list.append(filtered)

    gdf_time_filtered = pd.concat(gdf_time_filtered_list, ignore_index=True)
    return gdf_time_filtered.sort_values(by='year', ascending=True).reset_index(drop=True)

-----------------------------------------

Raster to gdf

In [ ]:
gdf_glacier = multi_raster_to_gdf(glacier_raster,1)

In [ ]:
gdf_water = multi_raster_to_gdf(water_rasters,2)

export gdf 

In [ ]:
pathg = ''
pathw = ''

In [ ]:
gdf_water.to_file(pathg)
gdf_glacier.to_file(pathw)

 importing polygonized shp

In [26]:
gdf_glacier=gpd.read_file(pathg)

In [27]:
gdf_water=gpd.read_file(pathw)

## Adding year to rgi

In [28]:
gdf_geom_RGI_2000 = add_year_column(gdf_geom_RGI_2000, 2000)
gdf_geom_RGI_2015 = add_year_column(gdf_geom_RGI_2015, 2015)
gdf_RGI_2000_FR = add_year_column(gdf_RGI_2000_FR, 2000)
gdf_RGI_2015_FR = add_year_column(gdf_RGI_2015_FR, 2015)

# I. FILTERING GLACIER

## filtering glacier outside RGI

In [29]:
glacier_inside_rgi_2000 = exclude_outside_RGI_glacier(gdf_glacier,gdf_geom_RGI_2000,2000)

In [30]:
glacier_inside_rgi_2015 = exclude_outside_RGI_glacier(gdf_glacier, geom_RGI_FR_2015,2015)

## Merging and conserving the filtered years

In [31]:
glacier_filtered_RGI_merged = []

# Ajout des années 1984 à 2000 depuis glacier_insidee_rgi_2000
for year in range(1985, 2001):
    glacier_filtered_RGI_merged.append(glacier_inside_rgi_2000[glacier_inside_rgi_2000["year"]==year])

# Ajout des années 2001 à 2015 depuis glacier_inside_rgi_2015
for year in range(2001, 2016):
    glacier_filtered_RGI_merged.append(glacier_inside_rgi_2015[glacier_inside_rgi_2015["year"]==year])

# Ajout des années 2016 à 2024 depuis glacier_inside
for year in range(2016, 2025):
    glacier_filtered_RGI_merged.append(gdf_glacier[gdf_glacier["year"]==year])

In [32]:
glacier_filtered_RGI_merged_gpd=pd.concat(glacier_filtered_RGI_merged, ignore_index=True)

## Time filtering (5 years)

In [33]:
time_filtered_glacier_decroissant=glacier_time_filtering_propre(glacier_filtered_RGI_merged_gpd)

In [34]:
time_filtered_glacier_propre_total=glacier_time_filtering_propre_total(glacier_filtered_RGI_merged_gpd)

In [35]:
# m=time_filtered_glacier_decroissant.explore(color='red',name='glacier')
# time_filtered_glacier_test.explore(color='blue',name='filter',m=m)
# folium.LayerControl().add_to(m)
# m

In [36]:
# time_filtered_glacier_decroissant.to_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_glacier_filtrage_5ans.shp")

In [37]:
# time_filtered_glacier_propre.to_file("C:/Users/ronir/Desktop/Google_Earth_Engine/Vdef/Vectors_filtered/gdf_glacier_filtrage_total.shp")

-----------------------------------------

# II. FILTERING WATER

### filtering water error in glacier

In [38]:
water_in_glacier, water_outside=filter_water_by_glacier_gpd(gdf_water, gdf_glacier)

### filtering water in RGI

In [39]:
water_outside_rgi_2000 = exclude_inside_RGI_water(water_outside, geom_RGI_FR_2000_to_export_year,2000)

In [40]:
water_outside_rgi_2015 = exclude_inside_RGI_water(water_outside, geom_RGI_FR_2015_to_export_year,2015)

Merging and conserving the filtered years

In [41]:
water_filtered_RGI_merged = []

# Ajout des années 1984 à 2000 depuis water_outside_rgi_2000
for year in range(1984, 2001):
    water_filtered_RGI_merged.append(water_outside_rgi_2000[water_outside_rgi_2000["year"]==year])

# Ajout des années 2001 à 2015 depuis water_outside_rgi_2015
for year in range(2001, 2016):
    water_filtered_RGI_merged.append(water_outside_rgi_2015[water_outside_rgi_2015["year"]==year])

# Ajout des années 2016 à 2024 depuis water_outside
for year in range(2016, 2025):
    water_filtered_RGI_merged.append(water_outside[water_outside["year"]==year])

In [42]:
water_filtered_RGI_merged_gpd=pd.concat(water_filtered_RGI_merged, ignore_index=True)

## Time filtering 

In [56]:
time_filtered_water_propre=water_time_filtering_propre_total(water_filtered_RGI_merged_gpd)

## Excluding water in glacier 2024

In [64]:
gdf_lake_filtered_excl_2024=exclude_water_inside_geometry(time_filtered_water_propre,time_filtered_glacier_propre_total[time_filtered_glacier_propre_total['year'] == 2024])

---------------------------------

# III. Applying Conditions


lake classification > glacier

In [72]:
lake_diss=gdf_lake_filtered_excl_2024.loc[gdf_lake_filtered_excl_2024['year']>1984].dissolve(by='year')

In [73]:
glacier_diss=time_filtered_glacier_propre_total.dissolve(by='year')

In [74]:
glacier_classified=glacier_diss.assign(geom_diff=glacier_diss.difference(lake_diss))

In [75]:
glacier_final=glacier_classified.drop(columns=["geometry"]).rename(columns={"geom_diff":"geometry"}).set_geometry("geometry").explode()

In [76]:
glacier_final=glacier_final.reset_index(drop=False)

## Associating each lake/glacier with the same id

Glacier_id d'après le lia_id

In [89]:
glacier_final_to_id = glacier_final.to_crs(gdf_lia_vdef.crs)

In [90]:
join_glac = glacier_final_to_id.sjoin(gdf_lia_vdef, how='inner', predicate='intersects')

# 3. Créer une colonne 'glacier_lia_id' dans glacier_final_1985 avec l'ID de gdf_lia
# join_glac contient à la fois les index de glacier_final_1985 (index_left) et de gdf_lia (index_right), et la colonne 'ID' de gdf_lia
glacier_def_clean_ided=glacier_final_to_id.copy()
glacier_def_clean_ided.loc[join_glac.index, 'glacier_lia_id'] = join_glac['index'].values


Pour les lacs, utilisation de la classif INRAE

## Merging the pixels of the same lakes using pyce

# Final Export

---------------------------------